<a href="https://colab.research.google.com/github/alaa1996001/Hello_world/blob/master/Copie_de_Fun_with_multithreading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Threads Harmony

### Packages to install

In [ ]:
!pip install --user --upgrade pillow
!pip install --user --upgrade pydub
!pip install --user --upgrade simpleaudio

### Imports

In [ ]:
from pydub import AudioSegment
from pydub.playback import play

import threading
import time

### Read & Play audio files

In [ ]:
# Change this variable to an actual mp3 file path
MP3_PATH = "test.mp3"
song = AudioSegment.from_mp3(MP3_PATH)
play(song) # Interrupt to stop music

### Playing on several threads

###### Without a lock (No harmony)

In [ ]:
MP3_PATH = "test.mp3"
song = AudioSegment.from_mp3(MP3_PATH)

def play_alternated_segment(song,i):
    while i*1000 < len(song):
        segment = song[i*1000:(i+10)*1000] #Decomposing the audio in segments
        i = i + 20
        play(segment)
    
segment_1 = song[:10*1000]
segment_2 = song[10*1000:20*1000]

t1 = threading.Thread(target=play_alternated_segment, args=(song,0))
t2 = threading.Thread(target=play_alternated_segment, args=(song,10))

t1.start();t2.start()

###### With a lock 

In [ ]:
MP3_PATH = "test.mp3"
song = AudioSegment.from_mp3(MP3_PATH)
lock = threading.RLock()

def play_alternated_segment(song,i,has_lock):
    if not has_lock:
        time.sleep(1)
    while i*1000 < len(song):
        lock.acquire()
        segment = song[i*1000:(i+10)*1000]
        i = i + 20
        play(segment)
        lock.release()
    
segment_1 = song[:10*1000]
segment_2 = song[10*1000:20*1000]

t1 = threading.Thread(target=play_alternated_segment, args=(song,0,True))
t2 = threading.Thread(target=play_alternated_segment, args=(song,10,False))

t1.start();t2.start()

### Rythming your multi threaded program

Lets say that we have 2 building compagnies: A and B. They all have separate work schedules composed with simple tasks, that the compagny can achieve by itself, or a complex task, that can only be achieved by getting help from another compagny. Each task have a random duration between 1 and 5 seconds.
If a compagny works on more than one task, it doubles the duration of all the current tasks it's working on.

In [ ]:
import threading
import random 
import time

class Task(object):
    def __init__(self,name,help_=None):
        self.name = name
        self.help = help_
        self.doubled = False
        self.duration = random.randint(1,5)
        
    def double_duration(self):
        if not self.doubled:
            self.duration = 2*self.duration; self.doubled = True
            print(self.name+" duration doubled")
            
    def get_duration(self):
        if self.doubled:
            return self.duration/2
        return self.duration
            
    def start(self):
        self.time_start = time.time()
        
    def ended(self):
        return time.time() > self.time_start+self.duration

class Compagny(threading.Thread):
    def __init__(self,name):
        super(Compagny,self).__init__(daemon=True)
        self.name = name
        self.working_on = []
        
    def set_schedule(self,schedule):
        self.schedule = schedule
        
    def work_on(self,task):
        self.working_on.append(task)
        task.start()
        print(self.name+" working on "+task.name+" (duration {}s)".format(task.duration))
        if len(self.working_on) > 1:
            [task.double_duration() for task in self.working_on]
        
    def complete_task(self,task):
        print(self.name+" has completed "+task.name)
        self.working_on.remove(task)

###### Without a lock 

In [ ]:
class Compagny_without_a_lock(Compagny):
    def run(self):
        start_time= time.time()
        for task in self.schedule:
            if task.help is not None:
                task.help.work_on(task)
            self.work_on(task)
            while len(self.working_on) > 0:
                for tsk in self.working_on:
                    if tsk.ended():
                        self.complete_task(tsk)
                time.sleep(0.1)
        self.duration = time.time()-start_time 

SCENARIO 1A: No help is needed for any task

In [ ]:
COMPAGNY_A_TASKS = [
    Task("Project A"),
    Task("Project B"),
    Task("Project C"),
]
COMPAGNY_B_TASKS = [
    Task("Project D"),
    Task("Project E"),
    Task("Project F"),
]

COMPAGNY_A = Compagny_without_a_lock("Compagny A");COMPAGNY_A.set_schedule(COMPAGNY_A_TASKS);
COMPAGNY_B = Compagny_without_a_lock("Compagny B");COMPAGNY_B.set_schedule(COMPAGNY_B_TASKS);

COMPAGNY_A.start();
COMPAGNY_B.start();

COMPAGNY_A.join()
COMPAGNY_B.join()

print('Compagny A minimum time possible: ',sum([task.get_duration() for task in COMPAGNY_A.schedule])+sum([task.get_duration() for task in COMPAGNY_B.schedule if task.help is COMPAGNY_A]))
print('Compagny A actual working time: ',COMPAGNY_A.duration)
print('Compagny B minimum time possible: ',sum([task.get_duration() for task in COMPAGNY_B.schedule])+sum([task.get_duration() for task in COMPAGNY_A.schedule if task.help is COMPAGNY_B]))
print('Compagny B actual working time: ',COMPAGNY_B.duration)

Compagny A working on Project A (duration 4s)
Compagny B working on Project D (duration 2s)
Compagny B has completed Project D
Compagny B working on Project E (duration 3s)
Compagny A has completed Project A
Compagny A working on Project B (duration 3s)
Compagny B has completed Project E
Compagny B working on Project F (duration 1s)
Compagny B has completed Project F
Compagny A has completed Project B
Compagny A working on Project C (duration 3s)
Compagny A has completed Project C
Compagny A minimum time possible:  10
Compagny A actual working time:  10.515083312988281
Compagny B minimum time possible:  6
Compagny B actual working time:  6.433267831802368


SCENARIO 1B: Simple Schedule with some tasks that needs collaboration

In [ ]:
COMPAGNY_A = Compagny_without_a_lock("Compagny A");
COMPAGNY_B = Compagny_without_a_lock("Compagny B");

COMPAGNY_A_TASKS = [
    Task("Project A"),
    Task("Project B",help_=COMPAGNY_B),
    Task("Project C"),
]
COMPAGNY_B_TASKS = [
    Task("Project D"),
    Task("Project E"),
    Task("Project F",help_=COMPAGNY_A),
]

COMPAGNY_A.set_schedule(COMPAGNY_A_TASKS);
COMPAGNY_B.set_schedule(COMPAGNY_B_TASKS);

COMPAGNY_A.start();
COMPAGNY_B.start();

COMPAGNY_A.join()
COMPAGNY_B.join()

print('Compagny A minimum time possible: ',sum([task.get_duration() for task in COMPAGNY_A.schedule])+sum([task.get_duration() for task in COMPAGNY_B.schedule if task.help is COMPAGNY_A]))
print('Compagny A actual working time: ',COMPAGNY_A.duration)
print('Compagny B minimum time possible: ',sum([task.get_duration() for task in COMPAGNY_B.schedule])+sum([task.get_duration() for task in COMPAGNY_A.schedule if task.help is COMPAGNY_B]))
print('Compagny B actual working time: ',COMPAGNY_B.duration)

Compagny A working on Project A (duration 2s)
Compagny B working on Project D (duration 4s)
Compagny A has completed Project A
Compagny B working on Project B (duration 1s)
Project D duration doubled
Project B duration doubled
Compagny A working on Project B (duration 2s)
Compagny B has completed Project B
Compagny A has completed Project B
Compagny A working on Project C (duration 5s)
Compagny B has completed Project D
Compagny B working on Project E (duration 3s)
Compagny A has completed Project C
Compagny B has completed Project E
Compagny A working on Project F (duration 1s)
Compagny B working on Project F (duration 1s)
Compagny B has completed Project F
Compagny A minimum time possible:  9.0
Compagny A actual working time:  9.494514226913452
Compagny B minimum time possible:  9.0
Compagny B actual working time:  12.450134515762329


###### With a lock 

In [ ]:
class Compagny_with_a_lock(Compagny):
    def __init__(self,name,lock):
        super(Compagny_with_a_lock,self).__init__(name)
        self.lock = lock
        
    def run(self):
        self.time_waiting = 0
        start_time= time.time()
        copy = self.schedule[:]
        while len(copy) > 0 or len(self.working_on) > 0:
            while len(self.working_on) > 0:
                for task in self.working_on:
                    if task.ended():
                        self.complete_task(task)
                time.sleep(0.1)
            try:
                self.lock.release()
            except:
                pass
            time.sleep(0.5) #Wait 0.5 to see if another compagny needs help
            self.lock.acquire()
            if len(self.working_on) == 0 and len(copy) > 0:
                task = copy.pop(0)
                if task.help is not None:
                    tm = time.time()
                    task.help.lock.acquire()
                    self.time_waiting += (time.time()-tm)
                    task.help.work_on(task)
                    task.help.lock.release()
                self.work_on(task)
        try:
            self.lock.release()
        except:
            pass
        self.duration = time.time()-start_time 

SCENARIO 2A: No help is needed for any task

In [ ]:
COMPAGNY_A_TASKS = [
    Task("Project A"),
    Task("Project B"),
    Task("Project C"),
]
COMPAGNY_B_TASKS = [
    Task("Project D"),
    Task("Project E"),
    Task("Project F"),
]

COMPAGNY_A = Compagny_with_a_lock("Compagny A",threading.Lock());COMPAGNY_A.set_schedule(COMPAGNY_A_TASKS);
COMPAGNY_B = Compagny_with_a_lock("Compagny B",threading.Lock());COMPAGNY_B.set_schedule(COMPAGNY_B_TASKS);

COMPAGNY_A.start();
COMPAGNY_B.start();

COMPAGNY_A.join()
COMPAGNY_B.join()

print('Compagny A minimum time possible: ',sum([task.get_duration() for task in COMPAGNY_A.schedule])+sum([task.get_duration() for task in COMPAGNY_B.schedule if task.help is COMPAGNY_A]))
print('Compagny A working time: ',COMPAGNY_A.duration-COMPAGNY_A.time_waiting)
print('Compagny A waiting time: ',COMPAGNY_A.time_waiting)
print('Compagny A total time: ',COMPAGNY_A.duration)
print('Compagny B minimum time possible: ',sum([task.get_duration() for task in COMPAGNY_B.schedule])+sum([task.get_duration() for task in COMPAGNY_A.schedule if task.help is COMPAGNY_B]))
print('Compagny B working time: ',COMPAGNY_B.duration-COMPAGNY_B.time_waiting)
print('Compagny B waiting time: ',COMPAGNY_B.time_waiting)
print('Compagny B total time: ',COMPAGNY_B.duration)

Compagny B working on Project D (duration 5s)Compagny A working on Project A (duration 3s)

Compagny A has completed Project A
Compagny A working on Project B (duration 2s)
Compagny B has completed Project D
Compagny A has completed Project BCompagny B working on Project E (duration 4s)

Compagny A working on Project C (duration 2s)
Compagny A has completed Project C
Compagny B has completed Project E
Compagny B working on Project F (duration 1s)
Compagny B has completed Project F
Compagny A minimum time possible:  7
Compagny A working time:  9.475216388702393
Compagny A waiting time:  0
Compagny A total time:  9.475216388702393
Compagny B minimum time possible:  10
Compagny B working time:  12.650744438171387
Compagny B waiting time:  0
Compagny B total time:  12.650744438171387


SCENARIO 2B: Simple Schedule with some tasks that needs collaboration

In [ ]:
COMPAGNY_A = Compagny_with_a_lock("Compagny A",threading.Lock());
COMPAGNY_B = Compagny_with_a_lock("Compagny B",threading.Lock());

COMPAGNY_A_TASKS = [
    Task("Project A"),
    Task("Project B",help_=COMPAGNY_B),
    Task("Project C"),
]
COMPAGNY_B_TASKS = [
    Task("Project D"),
    Task("Project E"),
    Task("Project F",help_=COMPAGNY_A),
]

COMPAGNY_A.set_schedule(COMPAGNY_A_TASKS);
COMPAGNY_B.set_schedule(COMPAGNY_B_TASKS);

COMPAGNY_A.start();
COMPAGNY_B.start();

COMPAGNY_A.join()
COMPAGNY_B.join()

print('Compagny A minimum time possible: ',sum([task.get_duration() for task in COMPAGNY_A.schedule])+sum([task.get_duration() for task in COMPAGNY_B.schedule if task.help is COMPAGNY_A]))
print('Compagny A working time: ',COMPAGNY_A.duration-COMPAGNY_A.time_waiting)
print('Compagny A waiting time: ',COMPAGNY_A.time_waiting)
print('Compagny A total time: ',COMPAGNY_A.duration)
print('Compagny B minimum time possible: ',sum([task.get_duration() for task in COMPAGNY_B.schedule])+sum([task.get_duration() for task in COMPAGNY_A.schedule if task.help is COMPAGNY_B]))
print('Compagny B working time: ',COMPAGNY_B.duration-COMPAGNY_B.time_waiting)
print('Compagny B waiting time: ',COMPAGNY_B.time_waiting)
print('Compagny B total time: ',COMPAGNY_B.duration)

Compagny B working on Project D (duration 4s)
Compagny A working on Project A (duration 5s)
Compagny B has completed Project D
Compagny B working on Project E (duration 4s)
Compagny A has completed Project A
Compagny B has completed Project E
Compagny B working on Project B (duration 4s)
Compagny A working on Project B (duration 4s)
Compagny A has completed Project B
Compagny B has completed Project B
Compagny A working on Project C (duration 2s)
Compagny A has completed Project C
Compagny A working on Project F (duration 3s)
Compagny B working on Project F (duration 3s)
Compagny A has completed Project F
Compagny B has completed Project F
Compagny A minimum time possible:  14
Compagny A working time:  16.681703805923462
Compagny A waiting time:  3.125770092010498
Compagny A total time:  19.80747389793396
Compagny B minimum time possible:  15
Compagny B working time:  17.70122003555298
Compagny B waiting time:  2.1218717098236084
Compagny B total time:  19.823091745376587


### Computing time loss per scenario

#### Scenario 1B

In [ ]:
import numpy  as np

time_losses = []
time_losses_perc = []
for i in range(0,10):
    print('iteration',i,'\n\n')
    COMPAGNY_A_TASKS = [
        Task("Project A"),
        Task("Project B"),
        Task("Project C"),
    ]
    COMPAGNY_B_TASKS = [
        Task("Project D"),
        Task("Project E"),
        Task("Project F"),
    ]

    COMPAGNY_A = Compagny_without_a_lock("Compagny A");COMPAGNY_A.set_schedule(COMPAGNY_A_TASKS);
    COMPAGNY_B = Compagny_without_a_lock("Compagny B");COMPAGNY_B.set_schedule(COMPAGNY_B_TASKS);

    COMPAGNY_A.start();
    COMPAGNY_B.start();

    COMPAGNY_A.join()
    COMPAGNY_B.join()
    
    min_a = sum([task.get_duration() for task in COMPAGNY_A.schedule])+sum([task.get_duration() for task in COMPAGNY_B.schedule if task.help is COMPAGNY_A])
    min_b = sum([task.get_duration() for task in COMPAGNY_B.schedule])+sum([task.get_duration() for task in COMPAGNY_A.schedule if task.help is COMPAGNY_B])
    time_losses.append(COMPAGNY_A.duration-min_a)
    time_losses.append(COMPAGNY_B.duration-min_b)
    time_losses_perc.append(100*(COMPAGNY_A.duration/min_a))
    time_losses_perc.append(100*(COMPAGNY_B.duration/min_b))
    
mean_loss = np.mean(time_losses)
mean_loss_p = np.mean(time_losses_perc)
print("Mean time lost: ",mean_loss,"s")
print("Mean time spent: ",str(mean_loss_p)[:5],"% of minimum time")

iteration 0 


Compagny A working on Project A (duration 4s)
Compagny B working on Project D (duration 1s)
Compagny B has completed Project D
Compagny B working on Project E (duration 2s)
Compagny B has completed Project E
Compagny B working on Project F (duration 5s)
Compagny A has completed Project A
Compagny A working on Project B (duration 5s)
Compagny B has completed Project F
Compagny A has completed Project B
Compagny A working on Project C (duration 3s)
Compagny A has completed Project C
iteration 1 


Compagny A working on Project A (duration 5s)
Compagny B working on Project D (duration 2s)
Compagny B has completed Project D
Compagny B working on Project E (duration 5s)
Compagny A has completed Project A
Compagny A working on Project B (duration 4s)
Compagny B has completed Project E
Compagny B working on Project F (duration 5s)
Compagny A has completed Project B
Compagny A working on Project C (duration 2s)
Compagny A has completed Project C
Compagny B has completed Project 

#### Scenario 1B

In [ ]:
import numpy  as np

time_losses = []
time_losses_perc = []
for i in range(0,10):
    print('iteration',i,'\n\n')
    COMPAGNY_A = Compagny_without_a_lock("Compagny A");
    COMPAGNY_B = Compagny_without_a_lock("Compagny B");

    COMPAGNY_A_TASKS = [
        Task("Project A"),
        Task("Project B",help_=COMPAGNY_B),
        Task("Project C"),
    ]
    COMPAGNY_B_TASKS = [
        Task("Project D"),
        Task("Project E"),
        Task("Project F",help_=COMPAGNY_A),
    ]

    COMPAGNY_A.set_schedule(COMPAGNY_A_TASKS);
    COMPAGNY_B.set_schedule(COMPAGNY_B_TASKS);

    COMPAGNY_A.start();
    COMPAGNY_B.start();

    COMPAGNY_A.join()
    COMPAGNY_B.join()
    
mean_loss = np.mean(time_losses)
mean_loss_p = np.mean(time_losses_perc)
print("Mean time lost: ",mean_loss,"s")
print("Mean time spent: ",str(mean_loss_p)[:5],"% of minimum time")

#### Scenario 2A

In [ ]:
import numpy  as np

time_losses = []
time_losses_perc = []
for i in range(0,10):
    print('iteration',i,'\n\n')
    COMPAGNY_A = Compagny_with_a_lock("Compagny A",threading.Lock());
    COMPAGNY_B = Compagny_with_a_lock("Compagny B",threading.Lock());

    COMPAGNY_A_TASKS = [
        Task("Project A"),
        Task("Project B"),
        Task("Project C"),
    ]
    COMPAGNY_B_TASKS = [
        Task("Project D"),
        Task("Project E"),
        Task("Project F"),
    ]

    COMPAGNY_A.set_schedule(COMPAGNY_A_TASKS);
    COMPAGNY_B.set_schedule(COMPAGNY_B_TASKS);

    COMPAGNY_A.start();
    COMPAGNY_B.start();

    COMPAGNY_A.join()
    COMPAGNY_B.join()
    
mean_loss = np.mean(time_losses)
mean_loss_p = np.mean(time_losses_perc)
print("Mean time lost: ",mean_loss,"s")
print("Mean time spent: ",str(mean_loss_p)[:5],"% of minimum time")

#### Scenario 1B

In [ ]:
import numpy  as np

time_losses = []
time_losses_perc = []
for i in range(0,10):
    print('iteration',i,'\n\n')
    COMPAGNY_A = Compagny_with_a_lock("Compagny A",threading.Lock());
    COMPAGNY_B = Compagny_with_a_lock("Compagny B",threading.Lock());

    COMPAGNY_A_TASKS = [
        Task("Project A"),
        Task("Project B",help_=COMPAGNY_B),
        Task("Project C"),
    ]
    COMPAGNY_B_TASKS = [
        Task("Project D"),
        Task("Project E"),
        Task("Project F",help_=COMPAGNY_A),
    ]

    COMPAGNY_A.set_schedule(COMPAGNY_A_TASKS);
    COMPAGNY_B.set_schedule(COMPAGNY_B_TASKS);

    COMPAGNY_A.start();
    COMPAGNY_B.start();

    COMPAGNY_A.join()
    COMPAGNY_B.join()
    
mean_loss = np.mean(time_losses)
mean_loss_p = np.mean(time_losses_perc)
print("Mean time lost: ",mean_loss,"s")
print("Mean time spent: ",str(mean_loss_p)[:5],"% of minimum time")

## Excercice

Figure out how to achieve synchronisation between compagnies using semaphores